In [1]:
from scipy.stats import ttest_ind
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.model_selection import GridSearchCV


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
#read the data
file_path = 'hpvdata.csv'
hpvdata = pd.read_csv(file_path)

/var/folders/b3/kkdtnh4d4gjgrk45d_yrf4100000gn/T/ipykernel_43331/3811312610.py:3: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  hpvdata = pd.read_csv(file_path)


In [3]:
#filter out all records that has NA values on VAX attitude
hpvdata_cleaned = hpvdata.dropna(subset=['HPV_VAX_attitu_s35'])

In [4]:
# Calculate the percentage of available (non-missing) values in each column
available_percentage = 100 - ((hpvdata_cleaned.isnull().sum() / len(hpvdata_cleaned)) * 100)
# Identify columns with more than 50% available values
columns_to_select = available_percentage[available_percentage > 50].index
# Select the columns from the DataFrame
hpvdata_cleaned = hpvdata_cleaned[columns_to_select]

In [6]:
hpvdata_cleaned['sec8_q89'].value_counts()

sec8_q89
 1.0      523
 2.0      402
 3.0       64
 4.0       42
-777.0      1
Name: count, dtype: int64

In [14]:
hpvdata_cleaned['sec5_q62'].value_counts()

sec5_q62
 3.0      901
 2.0       87
 1.0       43
-777.0      3
Name: count, dtype: int64

In [5]:
# Generate correlation matrix
corrmatrix  = hpvdata_cleaned.corr()

ValueError: could not convert string to float: '   .'

In [ ]:
corrmatrix  = hpvdata_cleaned.corr()
corrmatrix


In [ ]:
vars_ =  {}
for idx in corrmatrix.index:
    if abs(corrmatrix['HPV_VAX_attitu_s35'][idx])>0.7:
        vars_[idx] = corrmatrix['HPV_VAX_attitu_s35'][idx]

In [ ]:
# highly correlated variables
vars_

In [ ]:
hpvdata_cleaned['HPV_VAX_attitu_s35']
#higher the value, more negative

In [ ]:
df_low_attitude = hpvdata_cleaned[hpvdata_cleaned['HPV_VAX_attitu_s35']<=np.nanmedian(hpvdata_cleaned['HPV_VAX_attitu_s35'])]
df_high_attitude = hpvdata_cleaned[hpvdata_cleaned['HPV_VAX_attitu_s35']>np.nanmedian(hpvdata_cleaned['HPV_VAX_attitu_s35'])]


In [ ]:
df_high_attitude

In [ ]:
import matplotlib.pyplot as plt
df_high_attitude = df_high_attitude[df_high_attitude['sec1_q6']!=-999]
plt.hist(df_high_attitude['sec1_q6'], bins=6, range=[1,7],edgecolor='black')  # Adjust 'bins' parameter as needed
plt.title('Hidf_high_attitudestogram')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()

In [ ]:
print('median:',np.median(hpvdata_cleaned['HPV_VAX_attitu_s35']))
group1 = hpvdata_cleaned[hpvdata_cleaned['HPV_VAX_attitu_s35']<=14]
group2 = hpvdata_cleaned[hpvdata_cleaned['HPV_VAX_attitu_s35']>14]

In [ ]:
name = 'age'
print(name)
print('Score<=14')
print(np.mean(group1[name]))
print('Score>14')
print(np.mean(group2[name]))
print('pvalue')
print(ttest_ind(group1[name],group2[name],equal_var=True).pvalue)

In [ ]:
name = 'sec1_q8'
print('household income')
print('Score<=17')
print(np.mean(group1[group1[name]>-1][name]))
print('Score>17')
print(np.mean(group2[group2[name]>-1][name]))
print('pvalue')
print(ttest_ind(group1[group1[name]>-1][name],group2[group2[name]>-1][name],equal_var=True).pvalue)

In [ ]:
name = 'sec5_q61'
print('61.Have you ever talked about cervical cancer or HPV vaccine with other parents?')
print('Score<=17')
print(np.mean(group1[group1[name]>-1][name]))
print('Score>17')
print(np.mean(group2[group2[name]>-1][name]))
print('pvalue')
print(ttest_ind(group1[group1[name]>-1][name],group2[group2[name]>-1][name],equal_var=True).pvalue)

In [ ]:
name = 'sec5_q62'
print('62.How comfortable do you feel talking about cervical cancer or HPV vaccine with other parents?')
print('Score<=17')
print(np.mean(group1[group1[name]>-1][name]))
print('Score>17')
print(np.mean(group2[group2[name]>-1][name]))
print('pvalue')
print(ttest_ind(group1[group1[name]>-1][name],group2[group2[name]>-1][name],equal_var=True).pvalue)

In [ ]:
from scipy.stats import chi2_contingency
observed = np.zeros((2,2))
name = 'sec5_q61'
print('61.Have you ever talked about cervical cancer or HPV vaccine with other parents?')
observed[0,0] = group1[name].value_counts()[1]
observed[0,1] = group2[name].value_counts()[1]
observed[1,0] = group1[name].value_counts()[0]
observed[1,1] = group2[name].value_counts()[0]
chi2, p, dof, expected = chi2_contingency(observed)
print(observed)
print(p)

In [ ]:
name = 'sec1_q9'
print('residence')
observed = np.zeros((3,2))
observed[0,0] = group1[name].value_counts()[3]
observed[0,1] = group2[name].value_counts()[3]
observed[1,0] = group1[name].value_counts()[2]
observed[1,1] = group2[name].value_counts()[2]
observed[2,0] = group1[name].value_counts()[1]
observed[2,1] = group2[name].value_counts()[1]
chi2, p, dof, expected = chi2_contingency(observed)
print(observed)
print(p)

In [ ]:
from scipy.stats import chi2_contingency
name = 'sec5_q62'
print('62.How comfortable do you feel talking about cervical cancer or HPV vaccine with other parents?')
observed = np.zeros((3,2))
observed[0,0] = group1[name].value_counts()[3]
observed[0,1] = group2[name].value_counts()[3]
observed[1,0] = group1[name].value_counts()[2]
observed[1,1] = group2[name].value_counts()[2]
observed[2,0] = group1[name].value_counts()[1]
observed[2,1] = group2[name].value_counts()[1]
chi2, p, dof, expected = chi2_contingency(observed)
print(observed)
print(p)

In [ ]:
grouppos = hpvdata_cleaned[hpvdata_cleaned['HPV_VAX_attitu_s35']<12]
groupneu = hpvdata_cleaned[hpvdata_cleaned['HPV_VAX_attitu_s35']>=12]
groupneu = groupneu[groupneu['HPV_VAX_attitu_s35']<24]
groupneg = hpvdata_cleaned[hpvdata_cleaned['HPV_VAX_attitu_s35']>=24]


In [ ]:
len(groupneg)/len(hpvdata_cleaned)

In [ ]:
len(groupneu)/len(hpvdata_cleaned)

In [ ]:
len(grouppos)/len(hpvdata_cleaned)

In [ ]:
np.mean(groupneg['sec5_q61'])

In [ ]:
np.mean(groupneu['sec5_q61'])

In [ ]:
np.mean(grouppos['sec5_q61'])

In [ ]:
vars_ =  {}
for idx in corrmatrix.index:
    if abs(corrmatrix['sec5_q61'][idx])>0.1:
        vars_[idx] = corrmatrix['sec5_q61'][idx]

In [ ]:
vars_

In [ ]:
selected_data = hpvdata_cleaned[['HPV_VAX_attitu_s35','sec5_q61','age','sec1_q5','sec1_q6','sec1_q7','sec1_q8','sec1_q9','education_a']]
data = selected_data.dropna()
df_encoded = pd.get_dummies(data[['age','sec1_q5','sec1_q6','sec1_q7','sec1_q8','sec1_q9','education_a']], columns=['sec1_q5','sec1_q6','sec1_q7','sec1_q8','sec1_q9','education_a'])


In [ ]:
y = data['HPV_VAX_attitu_s35'].values.copy()
X = df_encoded.values.copy()
for i in range(len(y)):
    if y[i]>=20:
        y[i] = 1
    else:
        y[i] = 0

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
class_weights = {0: 1.0, 1: 1}
svc = SVC(kernel='rbf', probability=True, class_weight=class_weights)
param_grid = {'C': [0.1, 1, 10]}


In [ ]:
grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=5, scoring='roc_auc')
grid_search.fit(X_train, y_train)

In [ ]:
best_C = grid_search.best_params_['C']
best_svc = grid_search.best_estimator_

In [ ]:
best_svc

In [ ]:
y_pred = best_svc.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
y_pred

In [ ]:
y_scores = best_svc.predict_proba(X_test)[:, 1]

In [ ]:
# Calculate ROC curve and AUC
fpr, tpr, _ = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

